In [16]:
import gym
import torch
from pong.script.DQN import DQN # Get Network
from itertools import count
import numpy as np
import torchvision.transforms as T
from pong.script.GetScreen import get_screen
from PIL import Image

In [17]:
env = gym.make('Pong-v0').unwrapped
device = "cpu"
model = DQN().to(device)
model.load_state_dict(torch.load("../model/Pong_POLICY_20.pt", map_location=torch.device('cpu')))
model.eval()

DQN(
  (conv1): Conv2d(4, 16, kernel_size=(5, 5), stride=(2, 2))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (head): Linear(in_features=1568, out_features=3, bias=True)
)

In [18]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(80, interpolation=Image.CUBIC),
                    T.ToTensor()])

total_reward = 0
actions = np.zeros((3))

state = torch.cat((get_screen(env, resize, device),
                       get_screen(env, resize, device),
                       get_screen(env, resize, device),
                       get_screen(env, resize, device)), dim=1, out=None)

next_state = state.clone()
    
for t in count():
    # Select and perform an action
    action = model(state).max(1)[1].view(1, 1) + 1
    actions[action.item() - 1] += 1
    _, reward, done, _ = env.step(action.item())
    total_reward += reward
    reward = torch.tensor([reward], device=device)
    
    # Observe new state
    current_screen = get_screen(env, resize, device)
    if not done:
        next_state[:, 1:4, :, :] = next_state[:, 0:3, :, :].clone()
        next_state[:, 0, :, :] = current_screen
        # next_state = current_screen - last_screen
    else:
        next_state = None
    # Move to the next state
    state = next_state
    # Perform one step of the optimization (on the target network)
    if done:
        print("Total reward: ", total_reward, "Actions: ", actions)
        # plot_durations()
        break

Total reward:  -21.0 Actions:  [424. 475. 118.]
